In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

In [2]:
#Parametros de ejecucion del programa
nombreArchivoEstructuraDataset = "./Mapas de Transformación de Datos/Datos en Pasos Intermedios/Estructura del Dataset.txt"
nombreArchivoDatasetSinEtiquetar = "./Dataset sin Etiquetar.csv"
nombreArchivoDatasetEtiquetado = "./Dataset Etiquetado.csv"
nombreColumnaDuplicadaSalida = "Interested in buying a mobile home policy".replace(" ", "_")
nombreColumnaSalida = "Number of mobile home policies".replace(" ", "_")
nombreColumnaFase = "Phase"
etiquetaDatosEntrenamiento = "Training"
etiquetaDatosPrueba = "Test"
etiquetaVariablesCategoricas = "Categorical"

colorTextoVerde = (35, 155, 86)
colorTextoRojo = (176, 58, 46)

In [3]:
def printColouredText(text, foregroundColor = (0, 0, 0), backgroundColor = (255, 255, 255)):
    return "\033[48;2;{};{};{}m\033[38;2;{};{};{}m{}\033[38;2;0;0;0m\033[48;2;255;255;255m".format(backgroundColor[0], backgroundColor[1], backgroundColor[2], foregroundColor[0], foregroundColor[1], foregroundColor[2], text)

def buildCountingPivotTable(dataframeSource, valuesColumn, indexColumn, mainColumnToGroup, secondaryColumnToGroup):
    pivotTableWith2DetailLevels = pd.pivot_table(dataframeSource, values = valuesColumn, index = indexColumn, columns = [secondaryColumnToGroup, mainColumnToGroup], aggfunc = 'count')
    pivotTableWith2DetailLevels.fillna(0.0, inplace = True)
    pivotTableWith1DetailLevel = pd.pivot_table(dataframeSource, values = valuesColumn, index = indexColumn, columns = mainColumnToGroup, aggfunc = 'count')
    pivotTableWith1DetailLevel.fillna(0.0, inplace = True)
    pivotTableWith1DetailLevel["General Total"] = pivotTableWith1DetailLevel[pivotTableWith1DetailLevel.columns].sum(axis = 1)
    pivotTableWith1DetailLevel.rename(columns = {tableColumn: ("", tableColumn) for tableColumn in pivotTableWith1DetailLevel.columns}, inplace = True)
    fullPivotTable = pd.concat([pivotTableWith2DetailLevels, pivotTableWith1DetailLevel], axis = 1)
    
    suma = fullPivotTable.sum()
    suma.name = "General Total"
    pivotTableSummarizedByRows = pd.DataFrame(suma.transpose(), columns = fullPivotTable.columns, index = pd.Index([suma.name], name = indexColumn))
    pivotTableSummarizedByRows.loc[suma.name] = suma.tolist()
    fullPivotTable = pd.concat([fullPivotTable, pivotTableSummarizedByRows])
    
    for column in fullPivotTable.columns:
        fullPivotTable[column] = [int(value) for value in fullPivotTable[column].tolist()]
    return fullPivotTable

In [4]:
structureDatasetFile = pd.read_csv(nombreArchivoEstructuraDataset, sep = ";", decimal = ".")
fieldNamesList = structureDatasetFile["Field Name"].tolist()
respectiveVariableKindsList = structureDatasetFile["Variable Kind"].tolist()

In [5]:
notLabeledDatasetFile = pd.read_csv(nombreArchivoDatasetSinEtiquetar, sep = ";", decimal = ".")

print("Se ha extraído la información del archivo", printColouredText(nombreArchivoDatasetSinEtiquetar, colorTextoRojo))
print("Se tiene(n)", printColouredText(str(len(notLabeledDatasetFile.index)) + " dato(s)", colorTextoVerde))

Se ha extraído la información del archivo ./Dataset sin Etiquetar.csv
Se tiene(n) 9822 dato(s)


In [6]:
notLabeledDatasetFile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9822 entries, 0 to 9821
Data columns (total 87 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   Customer Subtype                                  9822 non-null   int64 
 1   Number of houses                                  9822 non-null   int64 
 2   Avg size househol                                 9822 non-null   int64 
 3   Avg age                                           9822 non-null   int64 
 4   Customer main type                                9822 non-null   int64 
 5   Roman catholic                                    9822 non-null   int64 
 6   Protestant                                        9822 non-null   int64 
 7   Other religion                                    9822 non-null   int64 
 8   No religion                                       9822 non-null   int64 
 9   Married                       

In [7]:
labeledDatasetFile = pd.read_csv(nombreArchivoDatasetEtiquetado, sep = ";", decimal = ".")
labeledDatasetFile[nombreColumnaDuplicadaSalida] = labeledDatasetFile[nombreColumnaSalida].tolist()
labeledDatasetFile.replace({nombreColumnaDuplicadaSalida: {0: "No", 1: "Yes"}}, inplace = True)

#Filtrado de informacion para separar el dataset de entrenamiento y el dataset de prueba del dataset completo
trainingDatasetFile = labeledDatasetFile[labeledDatasetFile[nombreColumnaFase].str.contains(etiquetaDatosEntrenamiento)]
testDatasetFile = labeledDatasetFile[labeledDatasetFile[nombreColumnaFase].str.contains(etiquetaDatosPrueba)]

print("Se ha extraído la información del archivo", printColouredText(nombreArchivoDatasetEtiquetado, colorTextoRojo))
print("Se tiene(n)", printColouredText(str(len(labeledDatasetFile.index)) + " dato(s)", colorTextoVerde))
print("Se tiene(n) de", printColouredText(str(len(trainingDatasetFile.index)) + " dato(s) de entrenamiento", colorTextoVerde))
print("Se tiene(n) de", printColouredText(str(len(testDatasetFile.index)) + " dato(s) de prueba", colorTextoVerde))

Se ha extraído la información del archivo ./Dataset Etiquetado.csv
Se tiene(n) 9822 dato(s)
Se tiene(n) de 5822 dato(s) de entrenamiento
Se tiene(n) de 4000 dato(s) de prueba


In [8]:
pivotTablesList = {}

prefix = "Label_"
for nombreColumna in notLabeledDatasetFile.columns:
    if nombreColumna != nombreColumnaSalida.replace("_", " ").replace("-", " ") and nombreColumna != nombreColumnaDuplicadaSalida.replace("_", " ").replace("-", " ") and nombreColumna != nombreColumnaFase:
        indexCol = fieldNamesList.index(nombreColumna)
        variableKind = respectiveVariableKindsList[indexCol]
        
        if variableKind == etiquetaVariablesCategoricas:
            print("pivotTablesList[\"" + nombreColumna + "\"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, \"" + prefix + nombreColumna.replace(" ", "_").replace("-", "_") + "\", nombreColumnaDuplicadaSalida, nombreColumnaFase)")
            print("display(pivotTablesList[\"" + nombreColumna + "\"])\n")
        else:
            print("pivotTablesList[\"" + nombreColumna + "\"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, \"" + nombreColumna.replace(" ", "_").replace("-", "_") + "\", nombreColumnaDuplicadaSalida, nombreColumnaFase)")
            print("display(pivotTablesList[\"" + nombreColumna + "\"])\n")
        #pivotTablesList[nombreColumna] = buildCountingPivotTable(notLabeledDatasetFile, nombreColumnaSalida, nombreColumna, nombreColumnaDuplicadaSalida, nombreColumnaFase)
        

#pivotTablesList[] = buildCountingPivotTable(notLabeledDatasetFile, nombreColumnaSalida, col, nombreColumnaDuplicadaSalida, nombreColumnaFase)

#display(pivotTable1)

pivotTablesList["Customer Subtype"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Customer_Subtype", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Customer Subtype"])

pivotTablesList["Number of houses"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_houses", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of houses"])

pivotTablesList["Avg size househol"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Avg_size_househol", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Avg size househol"])

pivotTablesList["Avg age"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Avg_age", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Avg age"])

pivotTablesList["Customer main type"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Customer_main_type", nombreC

In [9]:
pivotTablesList["Customer Subtype"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Customer_Subtype", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Customer Subtype"])

Phase                                      Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                  No  Yes   
Label_Customer_Subtype                                               
Affluent senior apartments                                 50    2   
Affluent young families                                    95   16   
Career and childcare                                      107   12   
Couples with teens 'Married with children'                209   16   
Dinki's (double income no kids)                            41    3   
Etnically diverse                                          23    2   
Family starters                                           144    9   
Fresh masters in the city                                   9    0   
High Income, expensive child                              111   13   
High status seniors                                       224   25   
Large family farms                                         71    0   
Large family, employed child                              173    9   
Large religous families                                   309   19   
Low income catholics                                      199    6   
Lower class large families                                764   46   
Middle class families                                     288   51   
Mixed apartment dwellers                                   94    4   
Mixed rurals                                              200    5   
Mixed seniors                                             176   10   
Mixed small town dwellers                                 122   10   
Modern, complete families                                 266   12   
Own home elderly                                           47    1   
Porchless seniors: no front yard                           84    2   
Religious elderly singles                                 114    4   
Residential elderly                                        25    0   
Senior cosmopolitans                                        5    0   
Seniors in apartments                                      49    1   
Single youth                                               19    0   
Stable family                                             156    9   
Students in apartments                                     16    0   
Suburban youth                                              3    0   
Traditional families                                      316   23   
Very Important Provincials                                 76    6   
Village families                                          206    8   
Young all american family                                 166   13   
Young and rising                                          247    4   
Young seniors in the city                                  80    2   
Young urban have-nots                                      15    0   
Young, low educated                                       175    5   
General Total                                            5474  348   

Phase                                      Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy              No  Yes    No  Yes   
Label_Customer_Subtype                                                      
Affluent senior apartments                             37    1    87    3   
Affluent young families                                71   12   166   28   
Career and childcare                                   76   14   183   26   
Couples with teens 'Married with children'            137   11   346   27   
Dinki's (double income no kids)                        26    2    67    5   
Etnically diverse                                      17    0    40    2   
Family starters                                       126    7   270   16   
Fresh masters in the city                               4    0    13    0   
High Income, expensive child                           81   13   192   26   
High status seniors                                   176    8   

In [10]:
pivotTablesList["Number of houses"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_houses", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of houses"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_houses                                                    
1                                                       4952  315   
2                                                        472   33   
3                                                         39    0   
4                                                          2    0   
5                                                          1    0   
6                                                          1    0   
7                                                          5    0   
8                                                          1    0   
10                                                         1    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_houses                                                           
1                                                   3437  211  8389  526   
2                                                    290   26   762   59   
3                                                     24    1    63    1   
4                                                      2    0     4    0   
5                                                      2    0     3    0   
6                                                      2    0     3    0   
7                                                      3    0     8    0   
8                                                      1    0     2    0   
10                                                     1    0     2    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_houses                                         
1                                                  8915  
2                                                   821  
3                                                    64  
4                                                     4  
5                                                     3  
6                                                     3  
7                                                     8  
8                                                     2  
10                                                    2  
General Total                                      9822

In [11]:
pivotTablesList["Avg size househol"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Avg_size_househol", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Avg size househol"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Avg_size_househol                                                   
1                                                        276    8   
2                                                       2016  115   
3                                                       2475  171   
4                                                        643   50   
5                                                         64    4   
6                                                          0    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Avg_size_househol                                                          
1                                                    165    3   441   11   
2                                                   1405   80  3421  195   
3                                                   1763  104  4238  275   
4                                                    391   48  1034   98   
5                                                     35    3    99    7   
6                                                      3    0     3    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Avg_size_househol                                        
1                                                   452  
2                                                  3616  
3                                                  4513  
4                                                  1132  
5                                                   106  
6                                                     3  
General Total                                      9822

In [12]:
pivotTablesList["Avg age"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Avg_age", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Avg age"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Label_Avg_age                                                       
20-30 years                                               73    1   
30-40 years                                             1365   87   
40-50 years                                             2817  183   
50-60 years                                             1009   64   
60-70 years                                              181   12   
70-80 years                                               29    1   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Label_Avg_age                                                              
20-30 years                                           30    0   103    1   
30-40 years                                          888   69  2253  156   
40-50 years                                         2034  120  4851  303   
50-60 years                                          663   41  1672  105   
60-70 years                                          128    8   309   20   
70-80 years                                           19    0    48    1   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Label_Avg_age                                            
20-30 years                                         104  
30-40 years                                        2409  
40-50 years                                        5154  
50-60 years                                        1777  
60-70 years                                         329  
70-80 years                                          49  
General Total                                      9822

In [13]:
pivotTablesList["Customer main type"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Customer_main_type", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Customer main type"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Label_Customer_main_type                                            
Average Family                                           827   59   
Career Loners                                             52    0   
Conservative families                                    625   42   
Cruising Seniors                                         201    4   
Driven Growers                                           436   66   
Family with grown ups                                   1474   89   
Farmers                                                  271    5   
Living well                                              554   15   
Retired and Religeous                                    530   20   
Successful hedonists                                     504   48   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Label_Customer_main_type                                                   
Average Family                                       577   50  1404  109   
Career Loners                                         27    0    79    0   
Conservative families                                411   33  1036   75   
Cruising Seniors                                     116    5   317    9   
Driven Growers                                       288   37   724  103   
Family with grown ups                               1069   62  2543  151   
Farmers                                              210    6   481   11   
Living well                                          368    3   922   18   
Retired and Religeous                                316   15   846   35   
Successful hedonists                                 380   27   884   75   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Label_Customer_main_type                                 
Average Family                                     1513  
Career Loners                                        79  
Conservative families                              1111  
Cruising Seniors                                    326  
Driven Growers                                      827  
Family with grown ups                              2694  
Farmers                                             492  
Living well                                         940  
Retired and Religeous                               881  
Successful hedonists                                959  
General Total                                      9822

In [14]:
pivotTablesList["Roman catholic"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Roman_catholic", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Roman catholic"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Label_Roman_catholic                                                
0%                                                      3051  177   
1 - 10%                                                 1492  107   
100%                                                       4    0   
11 - 23%                                                 679   54   
24 - 36%                                                 145    7   
37 - 49%                                                  65    1   
50 - 62%                                                  17    1   
63 - 75%                                                  12    1   
76 - 88%                                                   6    0   
89 - 99%                                                   3    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Label_Roman_catholic                                                       
0%                                                  2075  117  5126  294   
1 - 10%                                             1071   74  2563  181   
100%                                                   6    0    10    0   
11 - 23%                                             445   35  1124   89   
24 - 36%                                              86    5   231   12   
37 - 49%                                              54    3   119    4   
50 - 62%                                              11    1    28    2   
63 - 75%                                              10    2    22    3   
76 - 88%                                               4    1    10    1   
89 - 99%                                               0    0     3    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Label_Roman_catholic                                     
0%                                                 5420  
1 - 10%                                            2744  
100%                                                 10  
11 - 23%                                           1213  
24 - 36%                                            243  
37 - 49%                                            123  
50 - 62%                                             30  
63 - 75%                                             25  
76 - 88%                                             11  
89 - 99%                                              3  
General Total                                      9822

In [15]:
pivotTablesList["Protestant"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Protestant", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Protestant"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Label_Protestant                                                    
0%                                                        77    1   
1 - 10%                                                  129    5   
100%                                                     165    8   
11 - 23%                                                 372   24   
24 - 36%                                                 561   29   
37 - 49%                                                1520   87   
50 - 62%                                                1404   97   
63 - 75%                                                 675   39   
76 - 88%                                                 509   55   
89 - 99%                                                  62    3   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Label_Protestant                                                           
0%                                                    46    3   123    4   
1 - 10%                                               88    3   217    8   
100%                                                 114    3   279   11   
11 - 23%                                             248   22   620   46   
24 - 36%                                             413   19   974   48   
37 - 49%                                            1006   53  2526  140   
50 - 62%                                             962   70  2366  167   
63 - 75%                                             435   31  1110   70   
76 - 88%                                             401   30   910   85   
89 - 99%                                              49    4   111    7   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Label_Protestant                                         
0%                                                  127  
1 - 10%                                             225  
100%                                                290  
11 - 23%                                            666  
24 - 36%                                           1022  
37 - 49%                                           2666  
50 - 62%                                           2533  
63 - 75%                                           1180  
76 - 88%                                            995  
89 - 99%                                            118  
General Total                                      9822

In [16]:
pivotTablesList["Other religion"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Other_religion", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Other religion"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Label_Other_religion                                                
0%                                                      1873  130   
1 - 10%                                                 1921   93   
11 - 23%                                                1289   99   
24 - 36%                                                 239   18   
37 - 49%                                                 125    7   
50 - 62%                                                  27    1   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Label_Other_religion                                                       
0%                                                  1367   90  3240  220   
1 - 10%                                             1307   70  3228  163   
11 - 23%                                             846   60  2135  159   
24 - 36%                                             148   10   387   28   
37 - 49%                                              80    8   205   15   
50 - 62%                                              14    0    41    1   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Label_Other_religion                                     
0%                                                 3460  
1 - 10%                                            3391  
11 - 23%                                           2294  
24 - 36%                                            415  
37 - 49%                                            220  
50 - 62%                                             42  
General Total                                      9822

In [17]:
pivotTablesList["No religion"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "No_religion", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["No religion"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
No_religion                                                         
0                                                        419   37   
1                                                        209   21   
2                                                        986   69   
3                                                       1366   87   
4                                                       1264   70   
5                                                        912   51   
6                                                        213    4   
7                                                         92    9   
8                                                          5    0   
9                                                          8    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
No_religion                                                                
0                                                    292   25   711   62   
1                                                    142    6   351   27   
2                                                    675   48  1661  117   
3                                                    963   60  2329  147   
4                                                    855   56  2119  126   
5                                                    579   30  1491   81   
6                                                    169    8   382   12   
7                                                     73    5   165   14   
8                                                      8    0    13    0   
9                                                      6    0    14    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
No_religion                                              
0                                                   773  
1                                                   378  
2                                                  1778  
3                                                  2476  
4                                                  2245  
5                                                  1572  
6                                                   394  
7                                                   179  
8                                                    13  
9                                                    14  
General Total                                      9822

In [18]:
pivotTablesList["Married"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Married", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Married"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Married                                                             
0                                                         62    2   
1                                                         74    1   
2                                                        154    3   
3                                                        240    6   
4                                                        314   10   
5                                                        898   48   
6                                                       1101   71   
7                                                       1567  116   
8                                                        336   25   
9                                                        728   66   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Married                                                                    
0                                                     41    3   103    5   
1                                                     42    2   116    3   
2                                                     95    0   249    3   
3                                                    152    4   392   10   
4                                                    217    9   531   19   
5                                                    648   34  1546   82   
6                                                    795   48  1896  119   
7                                                   1045   72  2612  188   
8                                                    222   20   558   45   
9                                                    505   46  1233  112   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Married                                                  
0                                                   108  
1                                                   119  
2                                                   252  
3                                                   402  
4                                                   550  
5                                                  1628  
6                                                  2015  
7                                                  2800  
8                                                   603  
9                                                  1345  
General Total                                      9822

In [19]:
pivotTablesList["Living together"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Living_together", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Living together"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Living_together                                                     
0                                                       2287  161   
1                                                       1910  120   
2                                                       1016   59   
3                                                        152    7   
4                                                         77    1   
5                                                         18    0   
6                                                         13    0   
7                                                          1    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Living_together                                                            
0                                                   1626  111  3913  272   
1                                                   1288   84  3198  204   
2                                                    676   39  1692   98   
3                                                     99    2   251    9   
4                                                     53    2   130    3   
5                                                     14    0    32    0   
6                                                      5    0    18    0   
7                                                      1    0     2    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Living_together                                          
0                                                  4185  
1                                                  3402  
2                                                  1790  
3                                                   260  
4                                                   133  
5                                                    32  
6                                                    18  
7                                                     2  
General Total                                      9822

In [20]:
pivotTablesList["Other relation"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Other_relation", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Other relation"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Other_relation                                                      
0                                                       1080   93   
1                                                        506   33   
2                                                       1646  110   
3                                                       1078   74   
4                                                        622   26   
5                                                        260    6   
6                                                        175    4   
7                                                         63    1   
8                                                         21    0   
9                                                         23    1   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Other_relation                                                             
0                                                    743   65  1823  158   
1                                                    338   23   844   56   
2                                                   1116   72  2762  182   
3                                                    764   49  1842  123   
4                                                    474   18  1096   44   
5                                                    151    4   411   10   
6                                                    116    4   291    8   
7                                                     36    0    99    1   
8                                                     10    0    31    0   
9                                                     14    3    37    4   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Other_relation                                           
0                                                  1981  
1                                                   900  
2                                                  2944  
3                                                  1965  
4                                                  1140  
5                                                   421  
6                                                   299  
7                                                   100  
8                                                    31  
9                                                    41  
General Total                                      9822

In [21]:
pivotTablesList["Singles"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Singles", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Singles"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Singles                                                             
0                                                       1629  128   
1                                                        887   64   
2                                                       1172   75   
3                                                        807   41   
4                                                        498   21   
5                                                        246   13   
6                                                        123    4   
7                                                         66    1   
8                                                         23    1   
9                                                         23    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Singles                                                                    
0                                                   1068   91  2697  219   
1                                                    626   42  1513  106   
2                                                    850   46  2022  121   
3                                                    559   32  1366   73   
4                                                    354   17   852   38   
5                                                    152    5   398   18   
6                                                     93    2   216    6   
7                                                     37    3   103    4   
8                                                     12    0    35    1   
9                                                     11    0    34    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Singles                                                  
0                                                  2916  
1                                                  1619  
2                                                  2143  
3                                                  1439  
4                                                   890  
5                                                   416  
6                                                   222  
7                                                   107  
8                                                    36  
9                                                    34  
General Total                                      9822

In [22]:
pivotTablesList["Household without children"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Household_without_children", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Household without children"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Household_without_children                                          
0                                                        348   23   
1                                                        348   24   
2                                                       1003   57   
3                                                       1409   89   
4                                                       1367   88   
5                                                        576   30   
6                                                        294   27   
7                                                         88    8   
8                                                         13    1   
9                                                         28    1   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Household_without_children                                                 
0                                                    221   21   569   44   
1                                                    225   11   573   35   
2                                                    685   53  1688  110   
3                                                    965   54  2374  143   
4                                                    980   58  2347  146   
5                                                    416   21   992   51   
6                                                    181   14   475   41   
7                                                     61    3   149   11   
8                                                     12    1    25    2   
9                                                     16    2    44    3   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Household_without_children                               
0                                                   613  
1                                                   608  
2                                                  1798  
3                                                  2517  
4                                                  2493  
5                                                  1043  
6                                                   516  
7                                                   160  
8                                                    27  
9                                                    47  
General Total                                      9822

In [23]:
pivotTablesList["Household with children"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Household_with_children", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Household with children"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Household_with_children                                             
0                                                        148    5   
1                                                        282   10   
2                                                        596   39   
3                                                        914   59   
4                                                       1071   66   
5                                                       1048   58   
6                                                        727   56   
7                                                        328   23   
8                                                        188   18   
9                                                        172   14   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Household_with_children                                                    
0                                                     88    2   236    7   
1                                                    180    3   462   13   
2                                                    436   25  1032   64   
3                                                    590   33  1504   92   
4                                                    790   56  1861  122   
5                                                    722   41  1770   99   
6                                                    508   31  1235   87   
7                                                    223   27   551   50   
8                                                    123   12   311   30   
9                                                    102    8   274   22   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Household_with_children                                  
0                                                   243  
1                                                   475  
2                                                  1096  
3                                                  1596  
4                                                  1983  
5                                                  1869  
6                                                  1322  
7                                                   601  
8                                                   341  
9                                                   296  
General Total                                      9822

In [24]:
pivotTablesList["High level education"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "High_level_education", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["High level education"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
High_level_education                                                
0                                                       2056   91   
1                                                       1249   73   
2                                                       1074   70   
3                                                        508   39   
4                                                        289   37   
5                                                        166   21   
6                                                         59    8   
7                                                         45    6   
8                                                         20    2   
9                                                          8    1   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
High_level_education                                                       
0                                                   1406   68  3462  159   
1                                                    824   30  2073  103   
2                                                    718   59  1792  129   
3                                                    352   28   860   67   
4                                                    227   24   516   61   
5                                                    136   15   302   36   
6                                                     49    7   108   15   
7                                                     26    6    71   12   
8                                                     16    1    36    3   
9                                                      8    0    16    1   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
High_level_education                                     
0                                                  3621  
1                                                  2176  
2                                                  1921  
3                                                   927  
4                                                   577  
5                                                   338  
6                                                   123  
7                                                    83  
8                                                    39  
9                                                    17  
General Total                                      9822

In [25]:
pivotTablesList["Medium level education"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Medium_level_education", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Medium level education"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Medium_level_education                                              
0                                                        409   14   
1                                                        361   22   
2                                                        889   48   
3                                                       1250   80   
4                                                       1340   86   
5                                                        684   54   
6                                                        329   19   
7                                                        139   18   
8                                                         34    3   
9                                                         39    4   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Medium_level_education                                                     
0                                                    271   17   680   31   
1                                                    222   19   583   41   
2                                                    682   29  1571   77   
3                                                    949   53  2199  133   
4                                                    904   64  2244  150   
5                                                    453   36  1137   90   
6                                                    170    8   499   27   
7                                                     77    9   216   27   
8                                                     15    1    49    4   
9                                                     19    2    58    6   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Medium_level_education                                   
0                                                   711  
1                                                   624  
2                                                  1648  
3                                                  2332  
4                                                  2394  
5                                                  1227  
6                                                   526  
7                                                   243  
8                                                    53  
9                                                    64  
General Total                                      9822

In [26]:
pivotTablesList["Lower level education"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Lower_level_education", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Lower level education"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Lower_level_education                                               
0                                                        270   29   
1                                                        216   27   
2                                                        602   65   
3                                                        633   47   
4                                                        800   51   
5                                                        960   49   
6                                                        824   32   
7                                                        613   27   
8                                                        241   13   
9                                                        315    8   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Lower_level_education                                                      
0                                                    171   24   441   53   
1                                                    140   13   356   40   
2                                                    394   38   996  103   
3                                                    422   26  1055   73   
4                                                    558   39  1358   90   
5                                                    691   40  1651   89   
6                                                    592   24  1416   56   
7                                                    438   12  1051   39   
8                                                    158   12   399   25   
9                                                    198   10   513   18   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Lower_level_education                                    
0                                                   494  
1                                                   396  
2                                                  1099  
3                                                  1128  
4                                                  1448  
5                                                  1740  
6                                                  1472  
7                                                  1090  
8                                                   424  
9                                                   531  
General Total                                      9822

In [27]:
pivotTablesList["High status"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "High_status", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["High status"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
High_status                                                         
0                                                       1451   73   
1                                                       1184   61   
2                                                       1286   78   
3                                                        703   53   
4                                                        364   33   
5                                                        236   13   
6                                                        120   18   
7                                                         78   14   
8                                                         23    3   
9                                                         29    2   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
High_status                                                                
0                                                    995   57  2446  130   
1                                                    837   37  2021   98   
2                                                    854   60  2140  138   
3                                                    495   31  1198   84   
4                                                    225   19   589   52   
5                                                    151   15   387   28   
6                                                    103    7   223   25   
7                                                     62    5   140   19   
8                                                     18    3    41    6   
9                                                     22    4    51    6   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
High_status                                              
0                                                  2576  
1                                                  2119  
2                                                  2278  
3                                                  1282  
4                                                   641  
5                                                   415  
6                                                   248  
7                                                   159  
8                                                    47  
9                                                    57  
General Total                                      9822

In [28]:
pivotTablesList["Entrepreneur"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Entrepreneur", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Entrepreneur"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Entrepreneur                                                        
0                                                       3938  233   
1                                                       1120   82   
2                                                        322   26   
3                                                         34    3   
4                                                         11    1   
5                                                         49    3   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Entrepreneur                                                               
0                                                   2703  157  6641  390   
1                                                    755   52  1875  134   
2                                                    234   18   556   44   
3                                                     29    4    63    7   
4                                                      5    1    16    2   
5                                                     36    6    85    9   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Entrepreneur                                             
0                                                  7031  
1                                                  2009  
2                                                   600  
3                                                    70  
4                                                    18  
5                                                    94  
General Total                                      9822

In [29]:
pivotTablesList["Farmer"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Farmer", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Farmer"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Farmer                                                              
0                                                       3892  284   
1                                                        818   36   
2                                                        467   20   
3                                                        137    6   
4                                                         76    1   
5                                                         58    1   
6                                                         14    0   
7                                                          3    0   
8                                                          5    0   
9                                                          4    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Farmer                                                                     
0                                                   2613  196  6505  480   
1                                                    585   23  1403   59   
2                                                    313   15   780   35   
3                                                    112    1   249    7   
4                                                     64    3   140    4   
5                                                     49    0   107    1   
6                                                      7    0    21    0   
7                                                      0    0     3    0   
8                                                     10    0    15    0   
9                                                      9    0    13    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Farmer                                                   
0                                                  6985  
1                                                  1462  
2                                                   815  
3                                                   256  
4                                                   144  
5                                                   108  
6                                                    21  
7                                                     3  
8                                                    15  
9                                                    13  
General Total                                      9822

In [30]:
pivotTablesList["Middle management"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Middle_management", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Middle management"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Middle_management                                                   
0                                                        632   35   
1                                                        384   19   
2                                                       1406   85   
3                                                       1326   68   
4                                                        892   61   
5                                                        398   33   
6                                                        194   17   
7                                                        156   22   
8                                                         14    0   
9                                                         72    8   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Middle_management                                                          
0                                                    463   34  1095   69   
1                                                    282   14   666   33   
2                                                    961   56  2367  141   
3                                                    911   43  2237  111   
4                                                    580   40  1472  101   
5                                                    276   19   674   52   
6                                                    125   12   319   29   
7                                                    101   16   257   38   
8                                                     10    0    24    0   
9                                                     53    4   125   12   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Middle_management                                        
0                                                  1164  
1                                                   699  
2                                                  2508  
3                                                  2348  
4                                                  1573  
5                                                   726  
6                                                   348  
7                                                   295  
8                                                    24  
9                                                   137  
General Total                                      9822

In [31]:
pivotTablesList["Skilled labourers"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Skilled_labourers", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Skilled labourers"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Skilled_labourers                                                   
0                                                       1090   77   
1                                                        838   83   
2                                                       1296   86   
3                                                       1118   49   
4                                                        579   25   
5                                                        297   13   
6                                                        163    6   
7                                                         63    5   
8                                                         21    3   
9                                                          9    1   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Skilled_labourers                                                          
0                                                    761   67  1851  144   
1                                                    572   30  1410  113   
2                                                    886   59  2182  145   
3                                                    765   35  1883   84   
4                                                    385   17   964   42   
5                                                    197   17   494   30   
6                                                    122    8   285   14   
7                                                     48    3   111    8   
8                                                     12    2    33    5   
9                                                     14    0    23    1   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Skilled_labourers                                        
0                                                  1995  
1                                                  1523  
2                                                  2327  
3                                                  1967  
4                                                  1006  
5                                                   524  
6                                                   299  
7                                                   119  
8                                                    38  
9                                                    24  
General Total                                      9822

In [32]:
pivotTablesList["Unskilled labourers"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Unskilled_labourers", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Unskilled labourers"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Unskilled_labourers                                                 
0                                                        890   78   
1                                                        904   76   
2                                                       1357   82   
3                                                       1059   50   
4                                                        734   38   
5                                                        316   15   
6                                                        116    6   
7                                                         64    2   
8                                                          8    1   
9                                                         26    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Unskilled_labourers                                                        
0                                                    613   55  1503  133   
1                                                    656   49  1560  125   
2                                                    964   57  2321  139   
3                                                    670   38  1729   88   
4                                                    493   24  1227   62   
5                                                    225    8   541   23   
6                                                     89    5   205   11   
7                                                     33    2    97    4   
8                                                      8    0    16    1   
9                                                     11    0    37    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Unskilled_labourers                                      
0                                                  1636  
1                                                  1685  
2                                                  2460  
3                                                  1817  
4                                                  1289  
5                                                   564  
6                                                   216  
7                                                   101  
8                                                    17  
9                                                    37  
General Total                                      9822

In [33]:
pivotTablesList["Social class A"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Social_class_A", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Social class A"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Social_class_A                                                      
0                                                       1654   84   
1                                                       1489   80   
2                                                       1131   67   
3                                                        637   48   
4                                                        232   29   
5                                                        113   14   
6                                                         84   12   
7                                                         66   13   
8                                                         12    1   
9                                                         56    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Social_class_A                                                             
0                                                   1069   64  2723  148   
1                                                   1018   39  2507  119   
2                                                    803   55  1934  122   
3                                                    451   32  1088   80   
4                                                    177   14   409   43   
5                                                     88   13   201   27   
6                                                     48    8   132   20   
7                                                     60    8   126   21   
8                                                      3    0    15    1   
9                                                     45    5   101    5   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Social_class_A                                           
0                                                  2871  
1                                                  2626  
2                                                  2056  
3                                                  1168  
4                                                   452  
5                                                   228  
6                                                   152  
7                                                   147  
8                                                    16  
9                                                   106  
General Total                                      9822

In [34]:
pivotTablesList["Social class B1"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Social_class_B1", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Social class B1"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Social_class_B1                                                     
0                                                       1281   72   
1                                                       1399   81   
2                                                       1682  101   
3                                                        710   65   
4                                                        279   19   
5                                                         74    4   
6                                                         20    5   
7                                                          5    0   
8                                                          7    1   
9                                                         17    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Social_class_B1                                                            
0                                                    866   56  2147  128   
1                                                   1013   56  2412  137   
2                                                   1163   63  2845  164   
3                                                    483   42  1193  107   
4                                                    150   11   429   30   
5                                                     47    4   121    8   
6                                                     22    6    42   11   
7                                                      4    0     9    0   
8                                                      5    0    12    1   
9                                                      9    0    26    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Social_class_B1                                          
0                                                  2275  
1                                                  2549  
2                                                  3009  
3                                                  1300  
4                                                   459  
5                                                   129  
6                                                    53  
7                                                     9  
8                                                    13  
9                                                    26  
General Total                                      9822

In [35]:
pivotTablesList["Social class B2"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Social_class_B2", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Social class B2"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Social_class_B2                                                     
0                                                        932   58   
1                                                        814   47   
2                                                       1576  100   
3                                                       1102   73   
4                                                        605   47   
5                                                        340   17   
6                                                         90    6   
7                                                          6    0   
8                                                          7    0   
9                                                          2    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Social_class_B2                                                            
0                                                    650   54  1582  112   
1                                                    537   36  1351   83   
2                                                   1042   60  2618  160   
3                                                    807   43  1909  116   
4                                                    426   26  1031   73   
5                                                    219   12   559   29   
6                                                     73    7   163   13   
7                                                      3    0     9    0   
8                                                      4    0    11    0   
9                                                      1    0     3    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Social_class_B2                                          
0                                                  1694  
1                                                  1434  
2                                                  2778  
3                                                  2025  
4                                                  1104  
5                                                   588  
6                                                   176  
7                                                     9  
8                                                    11  
9                                                     3  
General Total                                      9822

In [36]:
pivotTablesList["Social class C"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Social_class_C", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Social class C"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Social_class_C                                                      
0                                                        339   25   
1                                                        242   30   
2                                                        799   71   
3                                                       1029   61   
4                                                       1097   62   
5                                                       1122   46   
6                                                        458   29   
7                                                        207   10   
8                                                         63    8   
9                                                        118    6   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Social_class_C                                                             
0                                                    241   29   580   54   
1                                                    197    9   439   39   
2                                                    547   51  1346  122   
3                                                    711   44  1740  105   
4                                                    732   38  1829  100   
5                                                    748   37  1870   83   
6                                                    298   15   756   44   
7                                                    172    7   379   17   
8                                                     41    5   104   13   
9                                                     75    3   193    9   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Social_class_C                                           
0                                                   634  
1                                                   478  
2                                                  1468  
3                                                  1845  
4                                                  1929  
5                                                  1953  
6                                                   800  
7                                                   396  
8                                                   117  
9                                                   202  
General Total                                      9822

In [37]:
pivotTablesList["Social class D"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Social_class_D", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Social class D"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Social_class_D                                                      
0                                                       2419  188   
1                                                       1465   98   
2                                                        812   40   
3                                                        427   14   
4                                                        218    5   
5                                                         99    1   
6                                                         21    1   
7                                                         12    1   
8                                                          0    0   
9                                                          1    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Social_class_D                                                             
0                                                   1641  128  4060  316   
1                                                   1041   54  2506  152   
2                                                    556   26  1368   66   
3                                                    301   15   728   29   
4                                                    143   12   361   17   
5                                                     56    2   155    3   
6                                                     14    1    35    2   
7                                                      9    0    21    1   
8                                                      1    0     1    0   
9                                                      0    0     1    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Social_class_D                                           
0                                                  4376  
1                                                  2658  
2                                                  1434  
3                                                   757  
4                                                   378  
5                                                   158  
6                                                    37  
7                                                    22  
8                                                     1  
9                                                     1  
General Total                                      9822

In [38]:
pivotTablesList["Rented house"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Rented_house", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Rented house"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Rented_house                                                        
0                                                        855   94   
1                                                        391   37   
2                                                        679   38   
3                                                        554   39   
4                                                        491   26   
5                                                        494   25   
6                                                        359   23   
7                                                        406   19   
8                                                        516   16   
9                                                        729   31   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Rented_house                                                               
0                                                    643   71  1498  165   
1                                                    302   25   693   62   
2                                                    438   22  1117   60   
3                                                    352   16   906   55   
4                                                    367   24   858   50   
5                                                    330   14   824   39   
6                                                    252   22   611   45   
7                                                    271   14   677   33   
8                                                    331   11   847   27   
9                                                    476   19  1205   50   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Rented_house                                             
0                                                  1663  
1                                                   755  
2                                                  1177  
3                                                   961  
4                                                   908  
5                                                   863  
6                                                   656  
7                                                   710  
8                                                   874  
9                                                  1255  
General Total                                      9822

In [39]:
pivotTablesList["Home owners"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Home_owners", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Home owners"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Home_owners                                                         
0                                                        729   31   
1                                                        514   16   
2                                                        407   19   
3                                                        359   23   
4                                                        474   25   
5                                                        494   26   
6                                                        565   39   
7                                                        686   38   
8                                                        391   37   
9                                                        855   94   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Home_owners                                                                
0                                                    476   19  1205   50   
1                                                    330   11   844   27   
2                                                    272   14   679   33   
3                                                    252   22   611   45   
4                                                    322   13   796   38   
5                                                    367   25   861   51   
6                                                    358   16   923   55   
7                                                    439   22  1125   60   
8                                                    303   25   694   62   
9                                                    643   71  1498  165   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Home_owners                                              
0                                                  1255  
1                                                   871  
2                                                   712  
3                                                   656  
4                                                   834  
5                                                   912  
6                                                   978  
7                                                  1185  
8                                                   756  
9                                                  1663  
General Total                                      9822

In [40]:
pivotTablesList["1 car"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "1_car", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["1 car"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
1_car                                                               
0                                                         19    0   
1                                                         14    0   
2                                                         57    1   
3                                                        224    7   
4                                                        435   13   
5                                                       1151   59   
6                                                       1572   91   
7                                                       1294  119   
8                                                        242   19   
9                                                        466   39   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
1_car                                                                      
0                                                     11    0    30    0   
1                                                      6    0    20    0   
2                                                     44    0   101    1   
3                                                    162    7   386   14   
4                                                    281   11   716   24   
5                                                    854   42  2005  101   
6                                                   1094   65  2666  156   
7                                                    853   72  2147  191   
8                                                    167    7   409   26   
9                                                    290   34   756   73   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
1_car                                                    
0                                                    30  
1                                                    20  
2                                                   102  
3                                                   400  
4                                                   740  
5                                                  2106  
6                                                  2822  
7                                                  2338  
8                                                   435  
9                                                   829  
General Total                                      9822

In [41]:
pivotTablesList["2 cars"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "2_cars", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["2 cars"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
2_cars                                                              
0                                                       1747  107   
1                                                       1384   84   
2                                                       1636  112   
3                                                        363   22   
4                                                        282   19   
5                                                         53    3   
6                                                          8    1   
7                                                          1    0   
9                                                          0    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
2_cars                                                                     
0                                                   1151   73  2898  180   
1                                                    934   52  2318  136   
2                                                   1167   84  2803  196   
3                                                    239   14   602   36   
4                                                    218   12   500   31   
5                                                     46    3    99    6   
6                                                      5    0    13    1   
7                                                      1    0     2    0   
9                                                      1    0     1    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
2_cars                                                   
0                                                  3078  
1                                                  2454  
2                                                  2999  
3                                                   638  
4                                                   531  
5                                                   105  
6                                                    14  
7                                                     2  
9                                                     1  
General Total                                      9822

In [42]:
pivotTablesList["No car"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "No_car", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["No car"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
No_car                                                              
0                                                       1329  121   
1                                                        728   48   
2                                                       1517  108   
3                                                       1017   49   
4                                                        574   13   
5                                                        170    4   
6                                                         84    5   
7                                                         25    0   
8                                                         13    0   
9                                                         17    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
No_car                                                                     
0                                                    936   89  2265  210   
1                                                    526   25  1254   73   
2                                                    920   66  2437  174   
3                                                    769   36  1786   85   
4                                                    417   16   991   29   
5                                                    110    4   280    8   
6                                                     54    2   138    7   
7                                                     15    0    40    0   
8                                                      5    0    18    0   
9                                                     10    0    27    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
No_car                                                   
0                                                  2475  
1                                                  1327  
2                                                  2611  
3                                                  1871  
4                                                  1020  
5                                                   288  
6                                                   145  
7                                                    40  
8                                                    18  
9                                                    27  
General Total                                      9822

In [43]:
pivotTablesList["National Health Service"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "National_Health_Service", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["National Health Service"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
National_Health_Service                                             
0                                                         48    7   
1                                                         15    0   
2                                                        280   27   
3                                                        162   15   
4                                                        329   28   
5                                                        902   72   
6                                                        830   45   
7                                                       1424   87   
8                                                        671   28   
9                                                        813   39   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
National_Health_Service                                                    
0                                                     48    6    96   13   
1                                                      8    0    23    0   
2                                                    191   20   471   47   
3                                                    124   14   286   29   
4                                                    253   18   582   46   
5                                                    620   50  1522  122   
6                                                    551   25  1381   70   
7                                                    946   63  2370  150   
8                                                    464   15  1135   43   
9                                                    557   27  1370   66   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
National_Health_Service                                  
0                                                   109  
1                                                    23  
2                                                   518  
3                                                   315  
4                                                   628  
5                                                  1644  
6                                                  1451  
7                                                  2520  
8                                                  1178  
9                                                  1436  
General Total                                      9822

In [44]:
pivotTablesList["Private health insurance"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Private_health_insurance", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Private health insurance"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Private_health_insurance                                            
0                                                        813   39   
1                                                        671   28   
2                                                       1424   87   
3                                                        804   45   
4                                                        920   72   
5                                                        336   28   
6                                                        163   15   
7                                                        280   27   
8                                                         15    0   
9                                                         48    7   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Private_health_insurance                                                   
0                                                    557   27  1370   66   
1                                                    464   15  1135   43   
2                                                    946   63  2370  150   
3                                                    541   25  1345   70   
4                                                    626   50  1546  122   
5                                                    257   18   593   46   
6                                                    124   14   287   29   
7                                                    191   20   471   47   
8                                                      8    0    23    0   
9                                                     48    6    96   13   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Private_health_insurance                                 
0                                                  1436  
1                                                  1178  
2                                                  2520  
3                                                  1415  
4                                                  1668  
5                                                   639  
6                                                   316  
7                                                   518  
8                                                    23  
9                                                   109  
General Total                                      9822

In [45]:
pivotTablesList["Income <30.000"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Income_<30.000", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Income <30.000"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Income_<30.000                                                      
0                                                       1206   98   
1                                                        576   54   
2                                                       1003   91   
3                                                       1029   50   
4                                                        578   21   
5                                                        551   17   
6                                                        284    9   
7                                                        149    7   
8                                                         47    1   
9                                                         51    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Income_<30.000                                                             
0                                                    788   72  1994  170   
1                                                    397   33   973   87   
2                                                    743   56  1746  147   
3                                                    706   41  1735   91   
4                                                    371   11   949   32   
5                                                    394    9   945   26   
6                                                    211    8   495   17   
7                                                     98    7   247   14   
8                                                     28    0    75    1   
9                                                     26    1    77    1   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Income_<30.000                                           
0                                                  2164  
1                                                  1060  
2                                                  1893  
3                                                  1826  
4                                                   981  
5                                                   971  
6                                                   512  
7                                                   261  
8                                                    76  
9                                                    78  
General Total                                      9822

In [46]:
pivotTablesList["Income 30-45.000"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Income_30_45.000", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Income 30-45.000"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Income_30_45.000                                                    
0                                                        439   26   
1                                                        250   18   
2                                                        864   55   
3                                                       1073   74   
4                                                       1272   84   
5                                                        886   45   
6                                                        381   25   
7                                                        195   10   
8                                                         33    2   
9                                                         81    9   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Income_30_45.000                                                           
0                                                    292   25   731   51   
1                                                    208   10   458   28   
2                                                    611   39  1475   94   
3                                                    793   49  1866  123   
4                                                    798   53  2070  137   
5                                                    623   31  1509   76   
6                                                    247   18   628   43   
7                                                    124    8   319   18   
8                                                     26    1    59    3   
9                                                     40    4   121   13   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Income_30_45.000                                         
0                                                   782  
1                                                   486  
2                                                  1569  
3                                                  1989  
4                                                  2207  
5                                                  1585  
6                                                   671  
7                                                   337  
8                                                    62  
9                                                   134  
General Total                                      9822

In [47]:
pivotTablesList["Income 45-75.000"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Income_45_75.000", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Income 45-75.000"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Income_45_75.000                                                    
0                                                        853   38   
1                                                        633   24   
2                                                       1117   48   
3                                                       1131   84   
4                                                        946   88   
5                                                        457   41   
6                                                        114   11   
7                                                         86    7   
8                                                         50    3   
9                                                         87    4   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Income_45_75.000                                                           
0                                                    592   22  1445   60   
1                                                    455   20  1088   44   
2                                                    763   34  1880   82   
3                                                    759   54  1890  138   
4                                                    618   57  1564  145   
5                                                    300   30   757   71   
6                                                     97   10   211   21   
7                                                     75    2   161    9   
8                                                     41    1    91    4   
9                                                     62    8   149   12   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Income_45_75.000                                         
0                                                  1505  
1                                                  1132  
2                                                  1962  
3                                                  2028  
4                                                  1709  
5                                                   828  
6                                                   232  
7                                                   170  
8                                                    95  
9                                                   161  
General Total                                      9822

In [48]:
pivotTablesList["Income 75-122.000"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Income_75_122.000", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Income 75-122.000"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Income_75_122.000                                                   
0                                                       3094  152   
1                                                       1262   97   
2                                                        678   58   
3                                                        227   19   
4                                                        133   14   
5                                                         64    7   
6                                                          8    0   
7                                                          1    0   
8                                                          4    0   
9                                                          3    1   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Income_75_122.000                                                          
0                                                   2096  122  5190  274   
1                                                    843   49  2105  146   
2                                                    482   38  1160   96   
3                                                    169   14   396   33   
4                                                    120   11   253   25   
5                                                     44    4   108   11   
6                                                      2    0    10    0   
7                                                      1    0     2    0   
8                                                      3    0     7    0   
9                                                      2    0     5    1   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Income_75_122.000                                        
0                                                  5464  
1                                                  2251  
2                                                  1256  
3                                                   429  
4                                                   278  
5                                                   119  
6                                                    10  
7                                                     2  
8                                                     7  
9                                                     6  
General Total                                      9822

In [49]:
pivotTablesList["Income >123.000"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Income_>123.000", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Income >123.000"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Income_>123.000                                                     
0                                                       4611  289   
1                                                        713   50   
2                                                         88    8   
3                                                         35    1   
4                                                         24    0   
5                                                          1    0   
6                                                          0    0   
7                                                          1    0   
9                                                          1    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Income_>123.000                                                            
0                                                   3155  198  7766  487   
1                                                    475   31  1188   81   
2                                                     85    7   173   15   
3                                                     29    1    64    2   
4                                                     13    1    37    1   
5                                                      3    0     4    0   
6                                                      2    0     2    0   
7                                                      0    0     1    0   
9                                                      0    0     1    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Income_>123.000                                          
0                                                  8253  
1                                                  1269  
2                                                   188  
3                                                    66  
4                                                    38  
5                                                     4  
6                                                     2  
7                                                     1  
9                                                     1  
General Total                                      9822

In [50]:
pivotTablesList["Average income"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Average_income", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Average income"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Average_income                                                      
0                                                         25    0   
1                                                         48    1   
2                                                        631   20   
3                                                       1863   69   
4                                                       1715  139   
5                                                        663   70   
6                                                        331   24   
7                                                        114   17   
8                                                         62    8   
9                                                         22    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Average_income                                                             
0                                                     13    0    38    0   
1                                                     26    3    74    4   
2                                                    449   10  1080   30   
3                                                   1238   62  3101  131   
4                                                   1127   82  2842  221   
5                                                    499   36  1162  106   
6                                                    263   28   594   52   
7                                                     86   11   200   28   
8                                                     47    4   109   12   
9                                                     14    2    36    2   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Average_income                                           
0                                                    38  
1                                                    78  
2                                                  1110  
3                                                  3232  
4                                                  3063  
5                                                  1268  
6                                                   646  
7                                                   228  
8                                                   121  
9                                                    38  
General Total                                      9822

In [51]:
pivotTablesList["Purchasing power class"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Purchasing_power_class", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Purchasing power class"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Purchasing_power_class                                              
1                                                        569   18   
2                                                        410   15   
3                                                       1453   71   
4                                                        856   46   
5                                                        553   30   
6                                                        835   66   
7                                                        407   67   
8                                                        391   35   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Purchasing_power_class                                                     
1                                                    337   14   906   32   
2                                                    302    4   712   19   
3                                                    982   50  2435  121   
4                                                    604   33  1460   79   
5                                                    356   25   909   55   
6                                                    650   36  1485  102   
7                                                    267   36   674  103   
8                                                    264   40   655   75   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Purchasing_power_class                                   
1                                                   938  
2                                                   731  
3                                                  2556  
4                                                  1539  
5                                                   964  
6                                                  1587  
7                                                   777  
8                                                   730  
General Total                                      9822

In [52]:
pivotTablesList["Contribution private third party insurance"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_private_third_party_insurance", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution private third party insurance"])

Phase                                            Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                        No  Yes   
Label_Contribution_private_third_party_insurance                           
0                                                              3335  147   
1 - 49                                                          193    8   
100 - 199                                                         9    2   
50 - 99                                                        1937  191   
General Total                                                  5474  348   

Phase                                            Phase 2 [Test]             \
Interested_in_buying_a_mobile_home_policy                    No  Yes    No   
Label_Contribution_private_third_party_insurance                             
0                                                          2321  100  5656   
1 - 49                                                      135    5   328   
100 - 199                                                     4    1    13   
50 - 99                                                    1302  132  3239   
General Total                                              3762  238  9236   

Phase                                                                
Interested_in_buying_a_mobile_home_policy         Yes General Total  
Label_Contribution_private_third_party_insurance                     
0                                                 247          5903  
1 - 49                                             13           341  
100 - 199                                           3            16  
50 - 99                                           323          3562  
General Total                                     586          9822

In [53]:
pivotTablesList["Contribution third party insurance (firms)"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_third_party_insurance_(firms)", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution third party insurance (firms)"])

Phase                                            Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                        No  Yes   
Label_Contribution_third_party_insurance_(firms)                           
0                                                              5397  343   
1 - 49                                                            7    0   
100 - 199                                                        20    3   
1000 - 4999                                                       4    0   
200 - 499                                                        17    0   
50 - 99                                                          28    2   
500 - 999                                                         1    0   
General Total                                                  5474  348   

Phase                                            Phase 2 [Test]             \
Interested_in_buying_a_mobile_home_policy                    No  Yes    No   
Label_Contribution_third_party_insurance_(firms)                             
0                                                          3712  236  9109   
1 - 49                                                        2    0     9   
100 - 199                                                    12    1    32   
1000 - 4999                                                   1    0     5   
200 - 499                                                     9    0    26   
50 - 99                                                      22    1    50   
500 - 999                                                     4    0     5   
General Total                                              3762  238  9236   

Phase                                                                
Interested_in_buying_a_mobile_home_policy         Yes General Total  
Label_Contribution_third_party_insurance_(firms)                     
0                                                 579          9688  
1 - 49                                              0             9  
100 - 199                                           4            36  
1000 - 4999                                         0             5  
200 - 499                                           0            26  
50 - 99                                             3            53  
500 - 999                                           0             5  
General Total                                     586          9822

In [54]:
pivotTablesList["Contribution third party insurane (agriculture)"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_third_party_insurane_(agriculture)", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution third party insurane (agriculture)"])

Phase                                              Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                          No  Yes   
Label_Contribution_third_party_insurane_(agricu...                           
0                                                                5357  345   
1 - 49                                                              0    0   
100 - 199                                                          55    2   
200 - 499                                                          59    1   
50 - 99                                                             3    0   
General Total                                                    5474  348   

Phase                                              Phase 2 [Test]             \
Interested_in_buying_a_mobile_home_policy                      No  Yes    No   
Label_Contribution_third_party_insurane_(agricu...                             
0                                                            3673  238  9030   
1 - 49                                                          2    0     2   
100 - 199                                                      35    0    90   
200 - 499                                                      48    0   107   
50 - 99                                                         4    0     7   
General Total                                                3762  238  9236   

Phase                                                                  
Interested_in_buying_a_mobile_home_policy           Yes General Total  
Label_Contribution_third_party_insurane_(agricu...                     
0                                                   583          9613  
1 - 49                                                0             2  
100 - 199                                             2            92  
200 - 499                                             1           108  
50 - 99                                               0             7  
General Total                                       586          9822

In [55]:
pivotTablesList["Contribution car policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_car_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution car policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Label_Contribution_car_policies                                     
0                                                       2773   72   
10.000 - 19.999                                            3    0   
1000 - 4999                                             2057  262   
20.000 - ?                                                 0    0   
200 - 499                                                  1    0   
500 - 999                                                599   14   
5000 - 9999                                               41    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Label_Contribution_car_policies                                            
0                                                   1914   66  4687  138   
10.000 - 19.999                                        2    0     5    0   
1000 - 4999                                         1433  158  3490  420   
20.000 - ?                                             1    0     1    0   
200 - 499                                              3    0     4    0   
500 - 999                                            388   12   987   26   
5000 - 9999                                           21    2    62    2   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Label_Contribution_car_policies                          
0                                                  4825  
10.000 - 19.999                                       5  
1000 - 4999                                        3910  
20.000 - ?                                            1  
200 - 499                                             4  
500 - 999                                          1013  
5000 - 9999                                          64  
General Total                                      9822

In [56]:
pivotTablesList["Contribution delivery van policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_delivery_van_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution delivery van policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Label_Contribution_delivery_van_policies                            
0                                                       5428  346   
1000 - 4999                                               33    2   
500 - 999                                                 10    0   
5000 - 9999                                                3    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Label_Contribution_delivery_van_policies                                   
0                                                   3719  237  9147  583   
1000 - 4999                                           35    1    68    3   
500 - 999                                              7    0    17    0   
5000 - 9999                                            1    0     4    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Label_Contribution_delivery_van_policies                 
0                                                  9730  
1000 - 4999                                          71  
500 - 999                                            17  
5000 - 9999                                           4  
General Total                                      9822

In [57]:
pivotTablesList["Contribution motorcycle/scooter policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_motorcycle/scooter_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution motorcycle/scooter policies"])

Phase                                          Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                      No  Yes   
Label_Contribution_motorcycle/scooter_policies                           
0                                                            5268  332   
100 - 199                                                       1    2   
1000 - 4999                                                    48    1   
200 - 499                                                     127    9   
500 - 999                                                      28    4   
5000 - 9999                                                     2    0   
General Total                                                5474  348   

Phase                                          Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy                  No  Yes    No  Yes   
Label_Contribution_motorcycle/scooter_policies                                  
0                                                        3629  231  8897  563   
100 - 199                                                   1    0     2    2   
1000 - 4999                                                29    1    77    2   
200 - 499                                                  67    4   194   13   
500 - 999                                                  36    2    64    6   
5000 - 9999                                                 0    0     2    0   
General Total                                            3762  238  9236  586   

Phase                                                         
Interested_in_buying_a_mobile_home_policy      General Total  
Label_Contribution_motorcycle/scooter_policies                
0                                                       9460  
100 - 199                                                  4  
1000 - 4999                                               79  
200 - 499                                                207  
500 - 999                                                 70  
5000 - 9999                                                2  
General Total                                           9822

In [58]:
pivotTablesList["Contribution lorry policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_lorry_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution lorry policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Label_Contribution_lorry_policies                                   
0                                                       5465  348   
1000 - 4999                                                7    0   
20.000 - ?                                                 1    0   
200 - 499                                                  1    0   
5000 - 9999                                                0    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Label_Contribution_lorry_policies                                          
0                                                   3757  238  9222  586   
1000 - 4999                                            3    0    10    0   
20.000 - ?                                             0    0     1    0   
200 - 499                                              0    0     1    0   
5000 - 9999                                            2    0     2    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Label_Contribution_lorry_policies                        
0                                                  9808  
1000 - 4999                                          10  
20.000 - ?                                            1  
200 - 499                                             1  
5000 - 9999                                           2  
General Total                                      9822

In [59]:
pivotTablesList["Contribution trailer policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_trailer_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution trailer policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Label_Contribution_trailer_policies                                 
0                                                       5415  342   
1 - 49                                                    18    1   
100 - 199                                                  6    0   
200 - 499                                                  1    0   
50 - 99                                                   33    5   
500 - 999                                                  1    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Label_Contribution_trailer_policies                                        
0                                                   3728  234  9143  576   
1 - 49                                                10    1    28    2   
100 - 199                                              3    0     9    0   
200 - 499                                              0    0     1    0   
50 - 99                                               21    3    54    8   
500 - 999                                              0    0     1    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Label_Contribution_trailer_policies                      
0                                                  9719  
1 - 49                                               30  
100 - 199                                             9  
200 - 499                                             1  
50 - 99                                              62  
500 - 999                                             1  
General Total                                      9822

In [60]:
pivotTablesList["Contribution tractor policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_tractor_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution tractor policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Label_Contribution_tractor_policies                                 
0                                                       5336  343   
100 - 199                                                 77    2   
1000 - 4999                                                8    1   
200 - 499                                                 27    0   
500 - 999                                                 26    2   
5000 - 9999                                                0    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Label_Contribution_tractor_policies                                        
0                                                   3662  235  8998  578   
100 - 199                                             60    3   137    5   
1000 - 4999                                            7    0    15    1   
200 - 499                                             18    0    45    0   
500 - 999                                             14    0    40    2   
5000 - 9999                                            1    0     1    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Label_Contribution_tractor_policies                      
0                                                  9576  
100 - 199                                           142  
1000 - 4999                                          16  
200 - 499                                            45  
500 - 999                                            42  
5000 - 9999                                           1  
General Total                                      9822

In [61]:
pivotTablesList["Contribution agricultural machines policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Contribution_agricultural_machines_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution agricultural machines policies"])

Phase                                       Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                   No  Yes   
Contribution_agricultural_machines_policies                           
0                                                         5453  348   
1                                                            0    0   
2                                                            4    0   
3                                                            6    0   
4                                                            8    0   
6                                                            3    0   
General Total                                             5474  348   

Phase                                       Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy               No  Yes    No  Yes   
Contribution_agricultural_machines_policies                                  
0                                                     3751  238  9204  586   
1                                                        1    0     1    0   
2                                                        3    0     7    0   
3                                                        2    0     8    0   
4                                                        3    0    11    0   
6                                                        2    0     5    0   
General Total                                         3762  238  9236  586   

Phase                                                      
Interested_in_buying_a_mobile_home_policy   General Total  
Contribution_agricultural_machines_policies                
0                                                    9790  
1                                                       1  
2                                                       7  
3                                                       8  
4                                                      11  
6                                                       5  
General Total                                        9822

In [62]:
pivotTablesList["Contribution moped policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_moped_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution moped policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Label_Contribution_moped_policies                                   
0                                                       5086  340   
100 - 199                                                276    6   
1000 - 4999                                                1    0   
200 - 499                                                 63    0   
50 - 99                                                   33    1   
500 - 999                                                 15    1   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Label_Contribution_moped_policies                                          
0                                                   3497  227  8583  567   
100 - 199                                            191    8   467   14   
1000 - 4999                                            1    0     2    0   
200 - 499                                             35    1    98    1   
50 - 99                                               27    2    60    3   
500 - 999                                             11    0    26    1   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Label_Contribution_moped_policies                        
0                                                  9150  
100 - 199                                           481  
1000 - 4999                                           2  
200 - 499                                            99  
50 - 99                                              63  
500 - 999                                            27  
General Total                                      9822

In [63]:
pivotTablesList["Contribution life insurances"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_life_insurances", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution life insurances"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Label_Contribution_life_insurances                                  
0                                                       5204  325   
1 - 49                                                     9    0   
10.000 - 19.999                                            1    0   
100 - 199                                                 78    6   
1000 - 4999                                               36    2   
20.000 - ?                                                 1    0   
200 - 499                                                 83   11   
50 - 99                                                   28    0   
500 - 999                                                 31    4   
5000 - 9999                                                3    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Label_Contribution_life_insurances                                         
0                                                   3568  211  8772  536   
1 - 49                                                 5    0    14    0   
10.000 - 19.999                                        0    0     1    0   
100 - 199                                             47   10   125   16   
1000 - 4999                                           25    2    61    4   
20.000 - ?                                             0    0     1    0   
200 - 499                                             69    9   152   20   
50 - 99                                               20    3    48    3   
500 - 999                                             27    3    58    7   
5000 - 9999                                            1    0     4    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Label_Contribution_life_insurances                       
0                                                  9308  
1 - 49                                               14  
10.000 - 19.999                                       1  
100 - 199                                           141  
1000 - 4999                                          65  
20.000 - ?                                            1  
200 - 499                                           172  
50 - 99                                              51  
500 - 999                                            65  
5000 - 9999                                           4  
General Total                                      9822

In [64]:
pivotTablesList["Contribution private accident insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_private_accident_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution private accident insurance policies"])

Phase                                              Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                          No  Yes   
Label_Contribution_private_accident_insurance_p...                           
0                                                                5444  347   
1 - 49                                                              3    0   
100 - 199                                                           4    0   
1000 - 4999                                                         2    0   
200 - 499                                                           3    0   
50 - 99                                                            17    1   
500 - 999                                                           1    0   
General Total                                                    5474  348   

Phase                                              Phase 2 [Test]             \
Interested_in_buying_a_mobile_home_policy                      No  Yes    No   
Label_Contribution_private_accident_insurance_p...                             
0                                                            3749  237  9193   
1 - 49                                                          2    1     5   
100 - 199                                                       3    0     7   
1000 - 4999                                                     0    0     2   
200 - 499                                                       1    0     4   
50 - 99                                                         6    0    23   
500 - 999                                                       1    0     2   
General Total                                                3762  238  9236   

Phase                                                                  
Interested_in_buying_a_mobile_home_policy           Yes General Total  
Label_Contribution_private_accident_insurance_p...                     
0                                                   584          9777  
1 - 49                                                1             6  
100 - 199                                             0             7  
1000 - 4999                                           0             2  
200 - 499                                             0             4  
50 - 99                                               1            24  
500 - 999                                             0             2  
General Total                                       586          9822

In [65]:
pivotTablesList["Contribution family accidents insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_family_accidents_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution family accidents insurance policies"])

Phase                                              Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                          No  Yes   
Label_Contribution_family_accidents_insurance_p...                           
0                                                                5442  342   
100 - 199                                                           9    4   
50 - 99                                                            23    2   
General Total                                                    5474  348   

Phase                                              Phase 2 [Test]             \
Interested_in_buying_a_mobile_home_policy                      No  Yes    No   
Label_Contribution_family_accidents_insurance_p...                             
0                                                            3729  231  9171   
100 - 199                                                      11    4    20   
50 - 99                                                        22    3    45   
General Total                                                3762  238  9236   

Phase                                                                  
Interested_in_buying_a_mobile_home_policy           Yes General Total  
Label_Contribution_family_accidents_insurance_p...                     
0                                                   573          9744  
100 - 199                                             8            28  
50 - 99                                               5            50  
General Total                                       586          9822

In [66]:
pivotTablesList["Contribution disability insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_disability_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution disability insurance policies"])

Phase                                            Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                        No  Yes   
Label_Contribution_disability_insurance_policies                           
0                                                              5455  344   
1000 - 4999                                                      15    4   
200 - 499                                                         1    0   
500 - 999                                                         1    0   
5000 - 9999                                                       2    0   
General Total                                                  5474  348   

Phase                                            Phase 2 [Test]             \
Interested_in_buying_a_mobile_home_policy                    No  Yes    No   
Label_Contribution_disability_insurance_policies                             
0                                                          3747  238  9202   
1000 - 4999                                                  13    0    28   
200 - 499                                                     0    0     1   
500 - 999                                                     0    0     1   
5000 - 9999                                                   2    0     4   
General Total                                              3762  238  9236   

Phase                                                                
Interested_in_buying_a_mobile_home_policy         Yes General Total  
Label_Contribution_disability_insurance_policies                     
0                                                 582          9784  
1000 - 4999                                         4            32  
200 - 499                                           0             1  
500 - 999                                           0             1  
5000 - 9999                                         0             4  
General Total                                     586          9822

In [67]:
pivotTablesList["Contribution fire policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_fire_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution fire policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Label_Contribution_fire_policies                                    
0                                                       2557  109   
1 - 49                                                   158    3   
10.000 - 19.999                                            1    0   
100 - 199                                                852   68   
1000 - 4999                                              152    3   
200 - 499                                               1075  151   
50 - 99                                                  529    6   
500 - 999                                                141    8   
5000 - 9999                                                9    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Label_Contribution_fire_policies                                           
0                                                   1726   72  4283  181   
1 - 49                                                84    0   242    3   
10.000 - 19.999                                        1    0     2    0   
100 - 199                                            584   37  1436  105   
1000 - 4999                                           93    4   245    7   
200 - 499                                            812  104  1887  255   
50 - 99                                              356   10   885   16   
500 - 999                                            103   11   244   19   
5000 - 9999                                            3    0    12    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Label_Contribution_fire_policies                         
0                                                  4464  
1 - 49                                              245  
10.000 - 19.999                                       2  
100 - 199                                          1541  
1000 - 4999                                         252  
200 - 499                                          2142  
50 - 99                                             901  
500 - 999                                           263  
5000 - 9999                                          12  
General Total                                      9822

In [68]:
pivotTablesList["Contribution surfboard policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_surfboard_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution surfboard policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Label_Contribution_surfboard_policies                               
0                                                       5472  347   
1 - 49                                                     1    1   
100 - 199                                                  1    0   
50 - 99                                                    0    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Label_Contribution_surfboard_policies                                      
0                                                   3758  236  9230  583   
1 - 49                                                 1    0     2    1   
100 - 199                                              0    0     1    0   
50 - 99                                                3    2     3    2   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Label_Contribution_surfboard_policies                    
0                                                  9813  
1 - 49                                                3  
100 - 199                                             1  
50 - 99                                               5  
General Total                                      9822

In [69]:
pivotTablesList["Contribution boat policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_boat_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution boat policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Label_Contribution_boat_policies                                    
0                                                       5454  335   
1 - 49                                                     2    3   
100 - 199                                                  3    2   
1000 - 4999                                                1    2   
200 - 499                                                  9    4   
50 - 99                                                    3    2   
500 - 999                                                  2    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Label_Contribution_boat_policies                                           
0                                                   3753  235  9207  570   
1 - 49                                                 1    0     3    3   
100 - 199                                              0    1     3    3   
1000 - 4999                                            0    0     1    2   
200 - 499                                              3    2    12    6   
50 - 99                                                3    0     6    2   
500 - 999                                              2    0     4    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Label_Contribution_boat_policies                         
0                                                  9777  
1 - 49                                                6  
100 - 199                                             6  
1000 - 4999                                           3  
200 - 499                                            18  
50 - 99                                               8  
500 - 999                                             4  
General Total                                      9822

In [70]:
pivotTablesList["Contribution bicycle policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_bicycle_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution bicycle policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Label_Contribution_bicycle_policies                                 
0                                                       5342  333   
1 - 49                                                   132   15   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Label_Contribution_bicycle_policies                                        
0                                                   3666  232  9008  565   
1 - 49                                                96    6   228   21   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Label_Contribution_bicycle_policies                      
0                                                  9573  
1 - 49                                              249  
General Total                                      9822

In [71]:
pivotTablesList["Contribution property insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_property_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution property insurance policies"])

Phase                                          Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                      No  Yes   
Label_Contribution_property_insurance_policies                           
0                                                            5434  343   
1 - 49                                                         15    3   
100 - 199                                                       6    0   
1000 - 4999                                                     1    0   
200 - 499                                                       3    0   
50 - 99                                                        14    2   
500 - 999                                                       1    0   
General Total                                                5474  348   

Phase                                          Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy                  No  Yes    No  Yes   
Label_Contribution_property_insurance_policies                                  
0                                                        3728  235  9162  578   
1 - 49                                                     15    1    30    4   
100 - 199                                                   1    0     7    0   
1000 - 4999                                                 1    0     2    0   
200 - 499                                                   5    0     8    0   
50 - 99                                                    12    2    26    4   
500 - 999                                                   0    0     1    0   
General Total                                            3762  238  9236  586   

Phase                                                         
Interested_in_buying_a_mobile_home_policy      General Total  
Label_Contribution_property_insurance_policies                
0                                                       9740  
1 - 49                                                    34  
100 - 199                                                  7  
1000 - 4999                                                2  
200 - 499                                                  8  
50 - 99                                                   30  
500 - 999                                                  1  
General Total                                           9822

In [72]:
pivotTablesList["Contribution social security insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_social_security_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution social security insurance policies"])

Phase                                              Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                          No  Yes   
Label_Contribution_social_security_insurance_po...                           
0                                                                5408  332   
100 - 199                                                          18    4   
200 - 499                                                          36    8   
50 - 99                                                            11    4   
500 - 999                                                           1    0   
General Total                                                    5474  348   

Phase                                              Phase 2 [Test]             \
Interested_in_buying_a_mobile_home_policy                      No  Yes    No   
Label_Contribution_social_security_insurance_po...                             
0                                                            3715  232  9123   
100 - 199                                                      12    0    30   
200 - 499                                                      18    6    54   
50 - 99                                                        16    0    27   
500 - 999                                                       1    0     2   
General Total                                                3762  238  9236   

Phase                                                                  
Interested_in_buying_a_mobile_home_policy           Yes General Total  
Label_Contribution_social_security_insurance_po...                     
0                                                   564          9687  
100 - 199                                             4            34  
200 - 499                                            14            68  
50 - 99                                               4            31  
500 - 999                                             0             2  
General Total                                       586          9822

In [73]:
pivotTablesList["Number of private third party insurance"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_private_third_party_insurance", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of private third party insurance"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_private_third_party_insurance                             
0                                                       3335  147   
1                                                       2133  201   
2                                                          6    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_private_third_party_insurance                                    
0                                                   2321  100  5656  247   
1                                                   1437  138  3570  339   
2                                                      4    0    10    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_private_third_party_insurance                  
0                                                  5903  
1                                                  3909  
2                                                    10  
General Total                                      9822

In [74]:
pivotTablesList["Number of third party insurance (firms)"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_third_party_insurance_(firms)", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of third party insurance (firms)"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_third_party_insurance_(firms)                             
0                                                       5397  343   
1                                                         76    5   
5                                                          1    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_third_party_insurance_(firms)                                    
0                                                   3712  236  9109  579   
1                                                     50    2   126    7   
5                                                      0    0     1    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_third_party_insurance_(firms)                  
0                                                  9688  
1                                                   133  
5                                                     1  
General Total                                      9822

In [75]:
pivotTablesList["Number of third party insurance (agriculture)"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_third_party_insurance_(agriculture)", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of third party insurance (agriculture)"])

Phase                                         Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                     No  Yes   
Number_of_third_party_insurance_(agriculture)                           
0                                                           5357  345   
1                                                            117    3   
General Total                                               5474  348   

Phase                                         Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy                 No  Yes    No  Yes   
Number_of_third_party_insurance_(agriculture)                                  
0                                                       3673  238  9030  583   
1                                                         89    0   206    3   
General Total                                           3762  238  9236  586   

Phase                                                        
Interested_in_buying_a_mobile_home_policy     General Total  
Number_of_third_party_insurance_(agriculture)                
0                                                      9613  
1                                                       209  
General Total                                          9822

In [76]:
pivotTablesList["Number of car policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_car_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of car policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_car_policies                                              
0                                                       2773   72   
1                                                       2475  237   
2                                                        208   38   
3                                                         11    1   
4                                                          5    0   
5                                                          0    0   
6                                                          1    0   
7                                                          1    0   
12                                                         0    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_car_policies                                                     
0                                                   1914   66  4687  138   
1                                                   1713  155  4188  392   
2                                                    122   16   330   54   
3                                                      8    1    19    2   
4                                                      3    0     8    0   
5                                                      1    0     1    0   
6                                                      0    0     1    0   
7                                                      0    0     1    0   
12                                                     1    0     1    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_car_policies                                   
0                                                  4825  
1                                                  4580  
2                                                   384  
3                                                    21  
4                                                     8  
5                                                     1  
6                                                     1  
7                                                     1  
12                                                    1  
General Total                                      9822

In [77]:
pivotTablesList["Number of delivery van policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_delivery_van_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of delivery van policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_delivery_van_policies                                     
0                                                       5428  346   
1                                                         38    2   
2                                                          4    0   
3                                                          3    0   
4                                                          1    0   
5                                                          0    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_delivery_van_policies                                            
0                                                   3719  237  9147  583   
1                                                     42    1    80    3   
2                                                      0    0     4    0   
3                                                      0    0     3    0   
4                                                      0    0     1    0   
5                                                      1    0     1    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_delivery_van_policies                          
0                                                  9730  
1                                                    83  
2                                                     4  
3                                                     3  
4                                                     1  
5                                                     1  
General Total                                      9822

In [78]:
pivotTablesList["Number of motorcycle/scooter policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_motorcycle/scooter_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of motorcycle/scooter policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_motorcycle/scooter_policies                               
0                                                       5268  332   
1                                                        196   15   
2                                                          9    1   
3                                                          0    0   
8                                                          1    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_motorcycle/scooter_policies                                      
0                                                   3629  231  8897  563   
1                                                    120    6   316   21   
2                                                     11    1    20    2   
3                                                      2    0     2    0   
8                                                      0    0     1    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_motorcycle/scooter_policies                    
0                                                  9460  
1                                                   337  
2                                                    22  
3                                                     2  
8                                                     1  
General Total                                      9822

In [79]:
pivotTablesList["Number of lorry policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_lorry_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of lorry policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_lorry_policies                                            
0                                                       5465  348   
1                                                          6    0   
2                                                          2    0   
3                                                          1    0   
4                                                          0    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_lorry_policies                                                   
0                                                   3757  238  9222  586   
1                                                      3    0     9    0   
2                                                      1    0     3    0   
3                                                      0    0     1    0   
4                                                      1    0     1    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_lorry_policies                                 
0                                                  9808  
1                                                     9  
2                                                     3  
3                                                     1  
4                                                     1  
General Total                                      9822

In [80]:
pivotTablesList["Number of trailer policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_trailer_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of trailer policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_trailer_policies                                          
0                                                       5415  342   
1                                                         53    6   
2                                                          4    0   
3                                                          2    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_trailer_policies                                                 
0                                                   3728  234  9143  576   
1                                                     33    4    86   10   
2                                                      1    0     5    0   
3                                                      0    0     2    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_trailer_policies                               
0                                                  9719  
1                                                    96  
2                                                     5  
3                                                     2  
General Total                                      9822

In [81]:
pivotTablesList["Number of tractor policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_tractor_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of tractor policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_tractor_policies                                          
0                                                       5336  343   
1                                                        101    4   
2                                                         28    1   
3                                                          3    0   
4                                                          6    0   
5                                                          0    0   
6                                                          0    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_tractor_policies                                                 
0                                                   3662  235  8998  578   
1                                                     76    3   177    7   
2                                                     17    0    45    1   
3                                                      4    0     7    0   
4                                                      0    0     6    0   
5                                                      1    0     1    0   
6                                                      2    0     2    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_tractor_policies                               
0                                                  9576  
1                                                   184  
2                                                    46  
3                                                     7  
4                                                     6  
5                                                     1  
6                                                     2  
General Total                                      9822

In [82]:
pivotTablesList["Number of agricultural machines policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_agricultural_machines_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of agricultural machines policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_agricultural_machines_policies                            
0                                                       5453  348   
1                                                         12    0   
2                                                          6    0   
3                                                          2    0   
4                                                          0    0   
6                                                          1    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_agricultural_machines_policies                                   
0                                                   3751  238  9204  586   
1                                                      9    0    21    0   
2                                                      1    0     7    0   
3                                                      0    0     2    0   
4                                                      1    0     1    0   
6                                                      0    0     1    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_agricultural_machines_policies                 
0                                                  9790  
1                                                    21  
2                                                     7  
3                                                     2  
4                                                     1  
6                                                     1  
General Total                                      9822

In [83]:
pivotTablesList["Number of moped policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_moped_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of moped policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_moped_policies                                            
0                                                       5086  340   
1                                                        374    8   
2                                                         14    0   
3                                                          0    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_moped_policies                                                   
0                                                   3497  227  8583  567   
1                                                    254   11   628   19   
2                                                     10    0    24    0   
3                                                      1    0     1    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_moped_policies                                 
0                                                  9150  
1                                                   647  
2                                                    24  
3                                                     1  
General Total                                      9822

In [84]:
pivotTablesList["Number of life insurances"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_life_insurances", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of life insurances"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_life_insurances                                           
0                                                       5204  325   
1                                                        165    8   
2                                                         90   10   
3                                                          9    2   
4                                                          5    3   
5                                                          0    0   
8                                                          1    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_life_insurances                                                  
0                                                   3568  211  8772  536   
1                                                    117   15   282   23   
2                                                     61    9   151   19   
3                                                     10    2    19    4   
4                                                      4    1     9    4   
5                                                      2    0     2    0   
8                                                      0    0     1    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_life_insurances                                
0                                                  9308  
1                                                   305  
2                                                   170  
3                                                    23  
4                                                    13  
5                                                     2  
8                                                     1  
General Total                                      9822

In [85]:
pivotTablesList["Number of private accident insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_private_accident_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of private accident insurance policies"])

Phase                                         Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                     No  Yes   
Number_of_private_accident_insurance_policies                           
0                                                           5444  347   
1                                                             30    1   
General Total                                               5474  348   

Phase                                         Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy                 No  Yes    No  Yes   
Number_of_private_accident_insurance_policies                                  
0                                                       3749  237  9193  584   
1                                                         13    1    43    2   
General Total                                           3762  238  9236  586   

Phase                                                        
Interested_in_buying_a_mobile_home_policy     General Total  
Number_of_private_accident_insurance_policies                
0                                                      9777  
1                                                        45  
General Total                                          9822

In [86]:
pivotTablesList["Number of family accidents insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_family_accidents_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of family accidents insurance policies"])

Phase                                         Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                     No  Yes   
Number_of_family_accidents_insurance_policies                           
0                                                           5442  342   
1                                                             32    6   
General Total                                               5474  348   

Phase                                         Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy                 No  Yes    No  Yes   
Number_of_family_accidents_insurance_policies                                  
0                                                       3729  231  9171  573   
1                                                         33    7    65   13   
General Total                                           3762  238  9236  586   

Phase                                                        
Interested_in_buying_a_mobile_home_policy     General Total  
Number_of_family_accidents_insurance_policies                
0                                                      9744  
1                                                        78  
General Total                                          9822

In [87]:
pivotTablesList["Number of disability insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_disability_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of disability insurance policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_disability_insurance_policies                             
0                                                       5455  344   
1                                                         15    4   
2                                                          4    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_disability_insurance_policies                                    
0                                                   3747  238  9202  582   
1                                                     15    0    30    4   
2                                                      0    0     4    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_disability_insurance_policies                  
0                                                  9784  
1                                                    34  
2                                                     4  
General Total                                      9822

In [88]:
pivotTablesList["Number of fire policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_fire_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of fire policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_fire_policies                                             
0                                                       2557  109   
1                                                       2785  232   
2                                                        119    7   
3                                                          7    0   
4                                                          3    0   
5                                                          2    0   
6                                                          0    0   
7                                                          1    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_fire_policies                                                    
0                                                   1726   72  4283  181   
1                                                   1939  160  4724  392   
2                                                     89    6   208   13   
3                                                      4    0    11    0   
4                                                      3    0     6    0   
5                                                      0    0     2    0   
6                                                      1    0     1    0   
7                                                      0    0     1    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_fire_policies                                  
0                                                  4464  
1                                                  5116  
2                                                   221  
3                                                    11  
4                                                     6  
5                                                     2  
6                                                     1  
7                                                     1  
General Total                                      9822

In [89]:
pivotTablesList["Number of surfboard policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_surfboard_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of surfboard policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_surfboard_policies                                        
0                                                       5472  347   
1                                                          2    1   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_surfboard_policies                                               
0                                                   3758  236  9230  583   
1                                                      4    2     6    3   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_surfboard_policies                             
0                                                  9813  
1                                                     9  
General Total                                      9822

In [90]:
pivotTablesList["Number of boat policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_boat_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of boat policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_boat_policies                                             
0                                                       5454  335   
1                                                         19   12   
2                                                          1    1   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_boat_policies                                                    
0                                                   3753  235  9207  570   
1                                                      7    2    26   14   
2                                                      2    1     3    2   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_boat_policies                                  
0                                                  9777  
1                                                    40  
2                                                     5  
General Total                                      9822

In [91]:
pivotTablesList["Number of bicycle policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_bicycle_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of bicycle policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_bicycle_policies                                          
0                                                       5342  333   
1                                                        101   10   
2                                                         30    4   
3                                                          1    1   
4                                                          0    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_bicycle_policies                                                 
0                                                   3666  232  9008  565   
1                                                     78    4   179   14   
2                                                     17    2    47    6   
3                                                      0    0     1    1   
4                                                      1    0     1    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_bicycle_policies                               
0                                                  9573  
1                                                   193  
2                                                    53  
3                                                     2  
4                                                     1  
General Total                                      9822

In [92]:
pivotTablesList["Number of property insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_property_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of property insurance policies"])

Phase                                     Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                 No  Yes   
Number_of_property_insurance_policies                               
0                                                       5434  343   
1                                                         39    5   
2                                                          1    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy             No  Yes    No  Yes   
Number_of_property_insurance_policies                                      
0                                                   3728  235  9162  578   
1                                                     34    3    73    8   
2                                                      0    0     1    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested_in_buying_a_mobile_home_policy General Total  
Number_of_property_insurance_policies                    
0                                                  9740  
1                                                    81  
2                                                     1  
General Total                                      9822

In [93]:
pivotTablesList["Number of social security insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_social_security_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of social security insurance policies"])

Phase                                        Phase 1 [Training]       \
Interested_in_buying_a_mobile_home_policy                    No  Yes   
Number_of_social_security_insurance_policies                           
0                                                          5408  332   
1                                                            65   16   
2                                                             1    0   
General Total                                              5474  348   

Phase                                        Phase 2 [Test]                  \
Interested_in_buying_a_mobile_home_policy                No  Yes    No  Yes   
Number_of_social_security_insurance_policies                                  
0                                                      3715  232  9123  564   
1                                                        47    6   112   22   
2                                                         0    0     1    0   
General Total                                          3762  238  9236  586   

Phase                                                       
Interested_in_buying_a_mobile_home_policy    General Total  
Number_of_social_security_insurance_policies                
0                                                     9687  
1                                                      134  
2                                                        1  
General Total                                         9822